In [11]:
# Scanorama batch effect correction
# See more at: https://github.com/brianhie/scanorama
# Hoa Tran 
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import time
from datetime import timedelta
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

scanpy==1.4+18.gaabe446 anndata==0.6.17 numpy==1.15.4 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [12]:
# Create folder to save the results 
import os
dirname = os.getcwd()
print(dirname)
data_dir = os.path.join('/home/hoa/hoatran/demo_normalization/dataset/dataset9_Human_cell_atlas/raw_data_python/')

if not os.path.exists('./results/'): os.makedirs('./results/')   
if not os.path.exists('./results/results_dataset9_HCA/'): os.makedirs('./results/results_dataset9_HCA/')    
save_dir = os.path.join(dirname, 'results/results_dataset9_HCA/')
print('Create a save dir: ',save_dir)
# Function to save figure as image in the figures folder
save_fig_dir='./figures/dataset9_HCA/'
if not os.path.exists('./figures/'): os.makedirs('./figures/')
if not os.path.exists(save_fig_dir): os.makedirs(save_fig_dir)
def save_images(filename, save_fig_dir):    
    outname = save_fig_dir + filename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python
Create a save dir:  /acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python/results/results_dataset9_HCA/


In [13]:
# def load_data(data_dir, myDataFn, mySampleFn,save_dir, savefn, saveh5ad=False):
#     myData = pd.read_csv(os.path.join(data_dir,myDataFn),header=0, index_col=0, sep='\t')
#     mySample = pd.read_csv(os.path.join(data_dir,mySampleFn),header=0, index_col=0, sep='\t')
#     adata = sc.AnnData(myData.values.T)
#     adata.obs_names = myData.keys()
#     adata.var_names = myData.index
# #     adata.obs['cell_type'] = mySample.loc[adata.obs_names,['celltype']]
#     adata.obs['batch'] = mySample.loc[adata.obs_names,['batch']]
# #     adata.obs['batch'] = adata.obs['batch'].astype('category')
#     adata.obs['batchlb'] = mySample.loc[adata.obs_names,['batchlb']]

#     # Save output into h5ad, lightweight, easy to access and load again
#     # Similar to rds format in R
#     if saveh5ad:
#         adata.write_h5ad(os.path.join(save_dir,savefn))
        
#     print(adata)
#     return adata

# myDataFn = 'filtered_genes_and_cells/HCA_genes_cells_filtered_filtered_UMI.txt'
# mySampleFn = 'HCA_genes_cells_filtered_filtered_cell_info_correct.txt'
# savefn = 'myRawData1.h5ad'
# adata = load_data(data_dir, myDataFn, mySampleFn, save_dir, savefn,saveh5ad=False)
# adata

In [14]:
savefn = 'HCA_genes_cells_filtered_filtered_UMI_adata.h5ad'
print(os.path.join(data_dir,savefn))
adata = sc.read_h5ad(os.path.join(data_dir,savefn))
print(adata)

/home/hoa/hoatran/demo_normalization/dataset/dataset9_Human_cell_atlas/raw_data_python/HCA_genes_cells_filtered_filtered_UMI_adata.h5ad
AnnData object with n_obs × n_vars = 621466 × 18969 
    obs: 'batch', 'batchlb'


In [15]:
# Already filtered
# sc.pp.filter_cells(adata, min_genes=300)
# sc.pp.filter_genes(adata, min_cells=10)

adata1 = adata[adata.obs['batch']==1,:].copy()
adata2 = adata[adata.obs['batch']==2,:].copy()
print(adata1)
print(adata2)
sum(adata2.var_names==adata1.var_names)

AnnData object with n_obs × n_vars = 321463 × 18969 
    obs: 'batch', 'batchlb'
AnnData object with n_obs × n_vars = 300003 × 18969 
    obs: 'batch', 'batchlb'
AnnData object with n_obs × n_vars = 0 × 18969 
    obs: 'batch', 'batchlb'


In [18]:
from scanorama import correct, visualize, process_data
from scanorama import dimensionality_reduce
import scanorama
adata_ls = [adata1, adata2]

In [19]:
# Batch correction.
# datasets = [df.values]
# List of datasets (matrices of cells-by-genes)
t1 = time.time()
corrected = scanorama.correct_scanpy(adata_ls, batch_size=100, return_dense=True, knn=20)
# corrected, genes = scanorama.correct(adata_ls, adata1.var_names,batch_size=30)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))

Found 18969 genes among all datasets
[[0.         0.29115948]
 [0.         0.        ]]
Processing datasets (0, 1)


Took 12:27:53.591624


In [20]:
adata = sc.AnnData(np.concatenate([corrected[0].X, corrected[1].X]))
print(adata)

AnnData object with n_obs × n_vars = 621466 × 18969 


In [21]:
adata.var_names = corrected[0].var_names
adata.obs_names = corrected[0].obs_names.tolist() + corrected[1].obs_names.tolist() 
# adata.obs['cell_type'] = corrected[0].obs['cell_type'].tolist() + corrected[1].obs['cell_type'].tolist() 
adata.obs['batch'] = corrected[0].obs['batch'].tolist() + corrected[1].obs['batch'].tolist() 
adata.obs['batchlb'] = corrected[0].obs['batchlb'].tolist() + corrected[1].obs['batchlb'].tolist() 
adata

AnnData object with n_obs × n_vars = 621466 × 18969 
    obs: 'batch', 'batchlb'

In [22]:
npcs = 20  
sc.tl.pca(adata, svd_solver='arpack', n_comps=npcs)  
adata.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
adata

AnnData object with n_obs × n_vars = 621466 × 18969 
    obs: 'batch', 'batchlb'
    uns: 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

In [23]:
def getExecutionTime(t1, t2, save_dir, usecase_name,filename):
    time_taken = (t2 - t1)
    time_taken_mins = divmod(time_taken, 60)
    time_taken_hours, rest = divmod( time_taken, 3600)
    hours_mins, hours_secs = divmod( rest, 60)
    print('Took seconds: '+str(timedelta(seconds=round(time_taken))))
    print('Took minutes: '+str(time_taken_mins))
    print('Took hours_minutes_seconds: ',str(time_taken_hours),str(hours_mins),str(hours_secs))
    
    

    data = {'use_case':usecase_name, 'exetime_secs':str(round(time_taken)),
           'exetimehours': str(time_taken_hours),
           'exetimemins': str(hours_mins),
           'exetimesecs':str(round(hours_secs))} 
    df = pd.DataFrame(data, index =['exetime'])
    print(df)
    df.to_csv(save_dir+filename) 

# Evaluation runtime of main batch effect removal function
filename = 'scanorama_exetime.csv'
usecase_name = 'scanorama_exetime' 
getExecutionTime(t1, t2, save_dir, usecase_name, filename)  # t1: start time, t2: end time  
print(save_dir)

Took seconds: 12:27:54
Took minutes: (747.0, 53.59162354469299)
Took hours_minutes_seconds:  12.0 27.0 53.59162354469299
                  use_case exetime_secs exetimehours exetimemins exetimesecs
exetime  scanorama_exetime        44874         12.0        27.0          54
/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python/results/results_dataset9_HCA/


In [24]:
def plotUMAP(adata, color_group, save_filename='umap', save_dir='', npcs=20, nb_neighbors=15, use_repx = False):
    
    # Run with all genes and entire expr matrix
    if use_repx:
        sc.pp.neighbors(adata, use_rep='X')
    else:    # Run umap using pcs vectors
        sc.pp.neighbors(adata,n_neighbors=nb_neighbors, n_pcs=npcs)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename, save_dir)

    
npcs = 20  # our pre-defined
perplex = 30  # our pre-defined
nb_neighbors = 15
color_group = ["batchlb",'batch'] 
save_fn_umap = 'scanorama_umap'
print(save_fig_dir)
plotUMAP(adata, color_group, save_fn_umap, save_fig_dir, npcs, nb_neighbors, False)
print('Save output of UMAP in :',save_fig_dir)

./figures/dataset9_HCA/
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:08:22.81) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
computing UMAP
    finished (0:22:17.32) --> added
    'X_umap', UMAP coordinates (adata.obsm)


... storing 'batchlb' as categorical


Save output of UMAP in : ./figures/dataset9_HCA/


In [26]:
def save_output_txt(adata, save_dir): 
    colnu = []
    for i in range(adata.obsm['X_umap'].shape[1]):
        colnu.append("UMAP"+str(i+1))
    df = pd.DataFrame(adata.obsm['X_umap'], columns=colnu, index=adata.obs_names)
    df['batch'] = pd.Series(adata.obs['batch'], index=adata.obs_names)
    df['batchlb'] = pd.Series(adata.obs['batchlb'], index=adata.obs_names)
#     df['celltype'] = pd.Series(adata.obs['cell_type'], index=adata.obs_names)
    df.to_csv(save_dir+'scanorama_umap.csv')  

#     # Save output of tsne for visualization
#     colnt = []
#     for i in range(adata.obsm['X_tsne'].shape[1]):
#         colnt.append("tSNE_"+str(i+1))

#     df = pd.DataFrame(adata.obsm['X_tsne'], columns=colnt, index=adata.obs_names)
#     df['batch'] = pd.Series(adata.obs['batch'], index=adata.obs_names)
#     df['batchlb'] = pd.Series(adata.obs['batchlb'], index=adata.obs_names)
# #     df['celltype'] = pd.Series(adata.obs['cell_type'], index=adata.obs_names)
#     df.to_csv(save_dir+'scanorama_tsne.csv') 

    # Save output of pca for evaluation ASW
    colnpc = []
    for i in range(20):
        colnpc.append("X_pca"+str(i+1))

    df = pd.DataFrame(adata.obsm['X_pca'][:, :20], columns=colnpc, index=adata.obs_names)
    df['batch'] = pd.Series(adata.obs['batch'], index=adata.obs_names)
    df['batchlb'] = pd.Series(adata.obs['batchlb'], index=adata.obs_names)
#     df['celltype'] = pd.Series(adata.obs['cell_type'], index=adata.obs_names)
    df.to_csv(save_dir+'scanorama_pca.csv')

# Apply to our data    
save_output_txt(adata, save_dir)
print('Save output of normalized data in :',save_dir)

Save output of normalized data in : /acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python/results/results_dataset9_HCA/


In [27]:
# # Function to plot TSNE
# def plotTSNE(adata, color_group, save_filename='tsne', save_dir='', n_pcs=20, perplex=30, use_repx = False):
    
#     # Run with all genes and entire matrix
#     if use_repx:
#         sc.tl.tsne(adata, perplexity=perplex, use_rep='X')
#     else:    # Run tsne using pcs vectors
#         sc.tl.tsne(adata, n_pcs=n_pcs, perplexity=perplex)
#     sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
#     save_images(save_filename, save_dir) 

# save_fn_tsne = 'scanorama_tsne'
# plotTSNE(adata, color_group, save_fn_tsne, save_fig_dir, npcs, perplex, False)
# print('Save output of t-SNE in :',save_fig_dir)

In [28]:
savefn = 'scanorama_normalized_adata.h5ad'
adata.write_h5ad(os.path.join(save_dir,savefn))
print('Save output of normalized data in :',save_dir)

Save output of normalized data in : /acrc/jinmiao/CJM_lab/hoatran/demo_normalization/demo_Scanorama/scanorama_python/results/results_dataset9_HCA/
